## Setup

### Setup

In [1]:
import os
from pathlib import Path
import pickle
import json
import warnings
import logging

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedStratifiedKFold
from utils import custom_reports_from_proba, custom_metrics_from_proba

In [2]:
warnings.filterwarnings("ignore", category=SyntaxWarning)

from tqdm import TqdmWarning
warnings.filterwarnings("ignore", category=TqdmWarning)

# Turn off pgmpy info messages
logging.getLogger('pgmpy').setLevel(logging.WARNING)

# Or turn off all info messages
logging.basicConfig(level=logging.WARNING)

In [3]:
project_path = Path(Path.cwd()).parent.parent

In [4]:
SEED = 7777

### Load Data

In [5]:
cancer_detection_path = project_path / "data/inputs/Lung Cancer Dataset.csv"
df_detection = pd.read_csv(cancer_detection_path)

df_detection.columns = [x for x in df_detection.columns.str.lower().str.replace(" ", "_")]

df_detection.head(10)

,age,gender,smoking,finger_discoloration,mental_stress,exposure_to_pollution,long_term_illness,energy_level,immune_weakness,breathing_issue,alcohol_consumption,throat_discomfort,oxygen_saturation,chest_tightness,family_history,smoking_family_history,stress_immune,pulmonary_disease
0,68,1,1,1,1,1,0,57.831178,0,0,1,1,95.977287,1,0,0,0,NO
1,81,1,1,0,0,1,1,47.694835,1,1,0,1,97.184483,0,0,0,0,YES
2,58,1,1,0,0,0,0,59.577435,0,1,1,0,94.974939,0,0,0,0,NO
3,44,0,1,0,1,1,0,59.785767,0,1,0,1,95.187900,0,0,0,0,YES
4,72,0,1,1,1,1,1,59.733941,0,1,0,1,93.503008,0,0,0,0,YES
5,37,1,1,1,1,1,1,57.684285,0,1,1,1,94.057151,1,0,0,0,YES
6,50,0,1,1,1,0,1,52.647022,1,1,1,0,96.773598,0,0,0,1,NO
7,68,0,1,1,1,0,1,53.306451,0,0,0,1,95.019018,0,0,0,0,NO
8,48,0,1,1,0,1,1,64.272789,1,1,0,1,98.539379,1,0,0,0,YES
9,52,0,0,0,1,1,1,58.319319,0,1,0,1,96.055097,0,0,0,0,NO


In [6]:
df_detection.shape

(5000, 18)

### Preprocessing

In [7]:
# Convert label column to numerical values
label_map_dict = {
    'NO': 0,
    'YES': 1
}

df_detection['pulmonary_disease'] = df_detection['pulmonary_disease'].map(label_map_dict)

In [8]:
# Convert binary columns to categorical
binary_columns = [
    'gender',
    'smoking',
    'finger_discoloration',
    'mental_stress',
    'exposure_to_pollution',
    'long_term_illness',
    'immune_weakness',
    'breathing_issue',
    'alcohol_consumption',
    'throat_discomfort',
    'chest_tightness',
    'family_history',
    'smoking_family_history',
    'stress_immune',
    'pulmonary_disease'
]

df_detection[binary_columns] = df_detection[binary_columns].astype('category')

In [9]:
df_X = df_detection.drop(columns=['pulmonary_disease'])
X = df_detection.drop(columns=['pulmonary_disease'])
y = df_detection['pulmonary_disease']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=SEED, stratify=y)

print("Train set shape:", X_train.shape)
print("Test set shape:", X_test.shape)

Train set shape: (4250, 17)
Test set shape: (750, 17)


In [10]:
with open("best_features.json", "r") as f:
    best_feature_set = json.load(f)

In [11]:
best_feature_mask = np.array(best_feature_set, dtype=bool)
print(best_feature_set)
print(best_feature_mask)

[1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1]
[ True False  True  True False  True False  True  True  True False  True
 False False  True  True  True]


In [12]:
X_train_fs = X_train.loc[:, best_feature_mask]
X_test_fs = X_test.loc[:, best_feature_mask]

In [13]:
train_data = X_train.copy()
train_data['pulmonary_disease'] = y_train.values

test_data = X_test.copy()
test_data['pulmonary_disease'] = y_test.values

In [14]:
train_data_fs = X_train_fs.copy()
train_data_fs['pulmonary_disease'] = y_train.values

test_data_fs = X_test_fs.copy()
test_data_fs['pulmonary_disease'] = y_test.values

## Modeling

### Naive Bayes

In [15]:
from implementations import run_naive_bayes_rskf_experiment
from config import CV_CONFIG, DISCRETIZATION_CONFIGS
from utils import custom_metrics_from_proba

INPUTS = {
    'og': {
        'X': X_train,
        'y': y_train,
        'discretization_cols': ['age', 'energy_level', 'oxygen_saturation']
    },
    'fs': {
        'X': X_train_fs,
        'y': y_train,
        'discretization_cols': ['age', 'energy_level']
    }
}

output_path = project_path / "data/outputs/naive_bayes.xlsx"


results_df = run_naive_bayes_rskf_experiment(
    inputs_config=INPUTS,
    discretization_configs=DISCRETIZATION_CONFIGS,
    cv_config=CV_CONFIG,
    output_path=output_path,
    custom_metrics_from_proba=custom_metrics_from_proba
)


Running: Input=og, Discretization={'method': 'quantile', 'n_bins': 3}
  Accuracy: 0.8389 ± 0.0161
  AUROC:    0.9023 ± 0.0128

Running: Input=og, Discretization={'method': 'quantile', 'n_bins': 4}
  Accuracy: 0.8391 ± 0.0152
  AUROC:    0.9022 ± 0.0129

Running: Input=og, Discretization={'method': 'quantile', 'n_bins': 5}
  Accuracy: 0.8423 ± 0.0156
  AUROC:    0.9028 ± 0.0128

Running: Input=og, Discretization={'method': 'quantile', 'n_bins': 7}
  Accuracy: 0.8413 ± 0.0149
  AUROC:    0.9032 ± 0.0130

Running: Input=og, Discretization={'method': 'quantile', 'n_bins': 9}
  Accuracy: 0.8415 ± 0.0169
  AUROC:    0.9034 ± 0.0126

Running: Input=og, Discretization={'method': 'uniform', 'n_bins': 3}
  Accuracy: 0.8303 ± 0.0154
  AUROC:    0.8965 ± 0.0125

Running: Input=og, Discretization={'method': 'uniform', 'n_bins': 4}
  Accuracy: 0.8408 ± 0.0161
  AUROC:    0.9024 ± 0.0126

Running: Input=og, Discretization={'method': 'uniform', 'n_bins': 5}
  Accuracy: 0.8363 ± 0.0165
  AUROC:    0.9

### TAN

In [16]:
from implementations import run_tan_rskf_experiment
from config import CV_CONFIG, DISCRETIZATION_CONFIGS
from utils import custom_metrics_from_proba

INPUTS = {
    'og': {
        'data': train_data,
        'target_col': 'pulmonary_disease',
        'discretization_cols': ['age', 'energy_level', 'oxygen_saturation']
    },
    'fs': {
        'data': train_data_fs,
        'target_col': 'pulmonary_disease',
        'discretization_cols': ['age', 'energy_level']
    }
}

output_path = project_path / "data/outputs/tan.xlsx"

results_df = run_tan_rskf_experiment(
    inputs_config=INPUTS,
    discretization_configs=DISCRETIZATION_CONFIGS,
    cv_config=CV_CONFIG,
    output_path=output_path,
    custom_metrics_from_proba=custom_metrics_from_proba
)


Running: Input=og, Discretization={'method': 'quantile', 'n_bins': 3}


Building tree: 100%|██████████| 153/153.0 [00:02<00:00, 58.60it/s]
Building tree: 100%|██████████| 153/153.0 [00:00<00:00, 2771.36it/s]
100%|██████████| 840/840 [00:17<00:00, 48.80it/s] 
Building tree: 100%|██████████| 153/153.0 [00:00<00:00, 4231.98it/s]
Building tree: 100%|██████████| 153/153.0 [00:00<00:00, 2420.71it/s]
100%|██████████| 845/845 [00:00<00:00, 2104.48it/s]
Building tree: 100%|██████████| 153/153.0 [00:00<00:00, 4960.45it/s]
Building tree: 100%|██████████| 153/153.0 [00:00<00:00, 2133.85it/s]
100%|██████████| 840/840 [00:00<00:00, 1959.64it/s]
Building tree: 100%|██████████| 153/153.0 [00:00<00:00, 3241.07it/s]
Building tree: 100%|██████████| 153/153.0 [00:00<00:00, 2759.73it/s]
100%|██████████| 838/838 [00:00<00:00, 2170.50it/s]
Building tree: 100%|██████████| 153/153.0 [00:00<00:00, 3223.57it/s]
Building tree: 100%|██████████| 153/153.0 [00:00<00:00, 2365.04it/s]
100%|██████████| 844/844 [00:00<00:00, 2103.82it/s]
Building tree: 100%|██████████| 153/153.0 [00:00<00:0

  Accuracy: 0.8715 ± 0.0109
  AUROC:    0.9147 ± 0.0106

Running: Input=og, Discretization={'method': 'quantile', 'n_bins': 4}


Building tree: 100%|██████████| 153/153.0 [00:00<00:00, 4143.36it/s]
Building tree: 100%|██████████| 153/153.0 [00:00<00:00, 2313.15it/s]
100%|██████████| 847/847 [00:00<00:00, 2254.55it/s]
Building tree: 100%|██████████| 153/153.0 [00:00<00:00, 3191.54it/s]
Building tree: 100%|██████████| 153/153.0 [00:00<00:00, 2357.36it/s]
100%|██████████| 847/847 [00:00<00:00, 1927.87it/s]
Building tree: 100%|██████████| 153/153.0 [00:00<00:00, 3362.19it/s]
Building tree: 100%|██████████| 153/153.0 [00:00<00:00, 2200.31it/s]
100%|██████████| 847/847 [00:00<00:00, 1968.37it/s]
Building tree: 100%|██████████| 153/153.0 [00:00<00:00, 4549.88it/s]
Building tree: 100%|██████████| 153/153.0 [00:00<00:00, 2149.26it/s]
100%|██████████| 847/847 [00:00<00:00, 2110.55it/s]
Building tree: 100%|██████████| 153/153.0 [00:00<00:00, 6113.97it/s]
Building tree: 100%|██████████| 153/153.0 [00:00<00:00, 2390.85it/s]
100%|██████████| 848/848 [00:00<00:00, 2204.19it/s]
Building tree: 100%|██████████| 153/153.0 [00:00<0

  Accuracy: 0.8719 ± 0.0117
  AUROC:    0.9136 ± 0.0104

Running: Input=og, Discretization={'method': 'quantile', 'n_bins': 5}


...

  Accuracy: 0.8725 ± 0.0108
  AUROC:    0.9137 ± 0.0104

Running: Input=fs, Discretization={'method': 'uniform', 'n_bins': 7}


Building tree: 100%|██████████| 66/66.0 [00:00<00:00, 11838.69it/s]
Building tree: 100%|██████████| 66/66.0 [00:00<00:00, 4795.57it/s]
100%|██████████| 701/701 [00:00<00:00, 2613.42it/s]
Building tree: 100%|██████████| 66/66.0 [00:00<00:00, 12137.68it/s]
Building tree: 100%|██████████| 66/66.0 [00:00<00:00, 6525.95it/s]
100%|██████████| 688/688 [00:00<00:00, 2307.44it/s]
Building tree: 100%|██████████| 66/66.0 [00:00<00:00, 12647.88it/s]
Building tree: 100%|██████████| 66/66.0 [00:00<00:00, 8726.29it/s]
100%|██████████| 711/711 [00:00<00:00, 2291.53it/s]
Building tree: 100%|██████████| 66/66.0 [00:00<00:00, 17528.28it/s]
Building tree: 100%|██████████| 66/66.0 [00:00<00:00, 5911.76it/s]
100%|██████████| 675/675 [00:00<00:00, 2419.39it/s]
Building tree: 100%|██████████| 66/66.0 [00:00<00:00, 11582.60it/s]
Building tree: 100%|██████████| 66/66.0 [00:00<00:00, 7340.28it/s]
100%|██████████| 695/695 [00:00<00:00, 2440.95it/s]
Building tree: 100%|██████████| 66/66.0 [00:00<00:00, 9510.57it/s

  Accuracy: 0.8744 ± 0.0107
  AUROC:    0.9126 ± 0.0087

Running: Input=fs, Discretization={'method': 'uniform', 'n_bins': 9}


Building tree: 100%|██████████| 66/66.0 [00:00<00:00, 7059.86it/s]
Building tree: 100%|██████████| 66/66.0 [00:00<00:00, 7296.94it/s]
100%|██████████| 741/741 [00:00<00:00, 2763.61it/s]
Building tree: 100%|██████████| 66/66.0 [00:00<00:00, 9143.05it/s]
Building tree: 100%|██████████| 66/66.0 [00:00<00:00, 9427.33it/s]
100%|██████████| 737/737 [00:00<00:00, 2970.94it/s]
Building tree: 100%|██████████| 66/66.0 [00:00<00:00, 13603.15it/s]
Building tree: 100%|██████████| 66/66.0 [00:00<00:00, 8247.40it/s]
100%|██████████| 746/746 [00:00<00:00, 2902.92it/s]
Building tree: 100%|██████████| 66/66.0 [00:00<00:00, 3885.25it/s]
Building tree: 100%|██████████| 66/66.0 [00:00<00:00, 9426.69it/s]
100%|██████████| 717/717 [00:00<00:00, 2864.58it/s]
Building tree: 100%|██████████| 66/66.0 [00:00<00:00, 21492.55it/s]
Building tree: 100%|██████████| 66/66.0 [00:00<00:00, 12801.11it/s]
100%|██████████| 743/743 [00:00<00:00, 2923.78it/s]
Building tree: 100%|██████████| 66/66.0 [00:00<00:00, 13943.69it/s]

  Accuracy: 0.8732 ± 0.0105
  AUROC:    0.9084 ± 0.0094

✓ Results saved to: c:\Users\jrafa\Desktop\12-sistemas-inteligentes\curso-si-puj\data\outputs\tan.xlsx


### BN

In [17]:
from implementations import run_bn_rskf_experiment
from config import CV_CONFIG, DISCRETIZATION_CONFIGS
from utils import custom_metrics_from_proba

INPUTS = {
    'og': {
        'data': train_data,
        'target_col': 'pulmonary_disease',
        'discretization_cols': ['age', 'energy_level', 'oxygen_saturation']
    },
    'fs': {
        'data': train_data_fs,
        'target_col': 'pulmonary_disease',
        'discretization_cols': ['age', 'energy_level']
    }
}

# Define structure learning methods to test (updated)
STRUCTURE_METHODS = ['hillclimb_k2', 'hillclimb_bic', 'hillclimb_bdeu']

output_path = project_path / "data/outputs/bayesian_network.xlsx"

results_df = run_bn_rskf_experiment(
    inputs_config=INPUTS,
    discretization_configs=DISCRETIZATION_CONFIGS,
    structure_methods=STRUCTURE_METHODS,
    cv_config=CV_CONFIG,
    output_path=output_path,
    custom_metrics_from_proba=custom_metrics_from_proba
)


Running: Input=og, Discretization={'method': 'quantile', 'n_bins': 3}, Structure=hillclimb_k2


100%|██████████| 267/267 [00:00<00:00, 2450.61it/s]


  Accuracy: 0.8707 ± 0.0090
  AUROC:    0.9215 ± 0.0097

Running: Input=og, Discretization={'method': 'quantile', 'n_bins': 3}, Structure=hillclimb_bic


100%|██████████| 267/267 [00:00<00:00, 2701.11it/s]


  Accuracy: 0.8657 ± 0.0107
  AUROC:    0.9187 ± 0.0105

Running: Input=og, Discretization={'method': 'quantile', 'n_bins': 3}, Structure=hillclimb_bdeu


...

  Accuracy: 0.8736 ± 0.0095
  AUROC:    0.9221 ± 0.0094

Running: Input=fs, Discretization={'method': 'uniform', 'n_bins': 9}, Structure=hillclimb_bic


100%|██████████| 314/314 [00:00<00:00, 1865.91it/s]


  Accuracy: 0.8650 ± 0.0118
  AUROC:    0.9190 ± 0.0100

Running: Input=fs, Discretization={'method': 'uniform', 'n_bins': 9}, Structure=hillclimb_bdeu


100%|██████████| 314/314 [00:00<00:00, 2269.56it/s]


  Accuracy: 0.8725 ± 0.0094
  AUROC:    0.9212 ± 0.0097

✓ Results saved to: c:\Users\jrafa\Desktop\12-sistemas-inteligentes\curso-si-puj\data\outputs\bayesian_network.xlsx


### BN (initial attempts)

In [ ]:
from pgmpy.estimators import BayesianEstimator

In [ ]:
from pgmpy.estimators import (
    PC, 
    HillClimbSearch, 
    BIC,
    MmhcEstimator,
    MaximumLikelihoodEstimator
)
from pgmpy.models import DiscreteBayesianNetwork
from pgmpy.inference import VariableElimination

In [ ]:
from discretization import discretize_data

columns_to_discretize = ['age', 'energy_level', 'oxygen_saturation']
train, test, _ = discretize_data(
    train_data,
    test_data,
    n_bins=5,
    strategy='uniform',
    columns_to_discretize=columns_to_discretize
)

In [ ]:
# ============================================
# 1️⃣ PC Algorithm (Constraint-based)
# ============================================
pc = PC(train)
dag_pc = pc.estimate(
    variant="stable",  # or "orig" or "parallel"
    significance_level=0.05  # adjust for conditional independence tests
)

model_pc = DiscreteBayesianNetwork(dag_pc.edges())
model_pc.fit(train, estimator=MaximumLikelihoodEstimator)

# Check which variables are in the model
print("Variables in learned model:", model_pc.nodes())
print("Edges in learned model:", model_pc.edges())

class_node = 'pulmonary_disease'

# ✅ Filter X_test to only include variables in the model
model_variables = set(model_pc.nodes())
available_features = list(model_variables - {class_node})

X_test = test[available_features]  # Only use features that exist in model
y_true = test[class_node]

y_pred_pc = model_pc.predict(X_test)
y_proba_pc = model_pc.predict_probability(X_test)

proba_col = f'{class_node}_1'
y_proba = y_proba_pc[proba_col].fillna(0.0).values

In [ ]:
metrics = custom_metrics_from_proba(y_true, y_proba)

In [ ]:
metrics

In [ ]:
# ============================================
# 2️⃣ Hill Climb + BIC (Score-based)
# ============================================
from pgmpy.estimators import HillClimbSearch, BIC

hc = HillClimbSearch(train)
bic = BIC(train)

dag_hc = hc.estimate(
    scoring_method=bic,
    max_indegree=4,
    max_iter=1000
)

model_hc = DiscreteBayesianNetwork(dag_hc.edges())
model_hc.fit(train, estimator=MaximumLikelihoodEstimator)

# Check which variables are in the model
print("Variables in learned model:", model_hc.nodes())
print("Edges in learned model:", model_hc.edges())

# ✅ Filter X_test to only include variables in the model
model_variables_hc = set(model_hc.nodes())
available_features_hc = list(model_variables_hc - {class_node})

X_test_hc = test[available_features_hc]  # Only use features that exist in model

y_pred_hc = model_hc.predict(X_test_hc)
y_proba_hc = model_hc.predict_probability(X_test_hc)

proba_col = f'{class_node}_1'
y_proba_hc = y_proba_hc[proba_col].fillna(0.0).values

In [ ]:
metrics = custom_metrics_from_proba(y_true, y_proba_hc)
metrics

In [ ]:
# ============================================
# 3️⃣ MMHC (Híbrido: restricción + puntuación)
# ============================================
from pgmpy.estimators import MmhcEstimator

mmhc = MmhcEstimator(train)
dag_mmhc = mmhc.estimate(
    significance_level=0.05,  # para la fase de restricciones (similar a PC)
    scoring_method=bic        # para la fase de puntuación (similar a Hill Climb)
)

model_mmhc = DiscreteBayesianNetwork(dag_mmhc.edges())
model_mmhc.fit(train, estimator=MaximumLikelihoodEstimator)

# Verificar qué variables están en el modelo
print("Variables en el modelo aprendido:", model_mmhc.nodes())
print("Aristas en el modelo aprendido:", model_mmhc.edges())

# ✅ Filtrar X_test para incluir solo las variables del modelo
model_variables_mmhc = set(model_mmhc.nodes())
available_features_mmhc = list(model_variables_mmhc - {class_node})

X_test_mmhc = test[available_features_mmhc]  # Usar solo las características presentes en el modelo

y_pred_mmhc = model_mmhc.predict(X_test_mmhc)
y_proba_mmhc = model_mmhc.predict_probability(X_test_mmhc)

proba_col = f'{class_node}_1'
y_proba_mmhc = y_proba_mmhc[proba_col].fillna(0.0).values

In [ ]:
metrics = custom_metrics_from_proba(y_true, y_proba_hc)
metrics

In [ ]:
from pgmpy.estimators import HillClimbSearch, BDeu, MaximumLikelihoodEstimator
from pgmpy.models import DiscreteBayesianNetwork

# Define the BDeu scoring method
bdeu = BDeu(train, equivalent_sample_size=10)

# Hill Climb search with BDeu score
hc = HillClimbSearch(train)
dag_hc_bdeu = hc.estimate(
    scoring_method=bdeu,
    max_indegree=4,   # limit parents per node (optional)
    max_iter=1000     # maximum iterations
)

# Create Bayesian Network model
model_hc_bdeu = DiscreteBayesianNetwork(dag_hc_bdeu.edges())

# Fit parameters using Maximum Likelihood Estimation
model_hc_bdeu.fit(train, estimator=MaximumLikelihoodEstimator)

# Inspect the learned structure
print("Variables in learned model:", model_hc_bdeu.nodes())
print("Edges in learned model:", model_hc_bdeu.edges())

# Define the class (target) variable
class_node = 'pulmonary_disease'

# Only use features that exist in the learned model
model_variables_bdeu = set(model_hc_bdeu.nodes())
available_features_bdeu = list(model_variables_bdeu - {class_node})

# Filter test data to available features
X_test_bdeu = test[available_features_bdeu]

# Predictions and probabilities
y_pred_bdeu = model_hc_bdeu.predict(X_test_bdeu)
y_proba_bdeu = model_hc_bdeu.predict_probability(X_test_bdeu)

# Extract probability for positive class (assuming binary target)
proba_col = f"{class_node}_1"
y_proba_bdeu = y_proba_bdeu[proba_col].fillna(0.0).values

print("\n✓ HillClimb + BDeu completed successfully!")
print(f"Predictions shape: {y_pred_bdeu.shape}")
print(f"Probabilities shape: {y_proba_bdeu.shape}")


In [ ]:
X_test = test[available_features_bdeu]  # Only use features that exist in model
y_true = test[class_node]

In [ ]:
metrics = custom_metrics_from_proba(y_true, y_proba_bdeu)
metrics

In [ ]:
from pgmpy.estimators import HillClimbSearch, BIC

# 1️⃣ Aprendizaje de estructura
hc = HillClimbSearch(train)
bic = BIC(train)

dag_hc = hc.estimate(
    scoring_method=bic,
    max_indegree=4,
    max_iter=1000
)

# 2️⃣ Aprendizaje de parámetros
model_hc_bdeu.fit(train, estimator=MaximumLikelihoodEstimator)

### TAN (Initial attempts)

In [ ]:
from pgmpy.estimators import TreeSearch, MaximumLikelihoodEstimator
from pgmpy.models import DiscreteBayesianNetwork
from pgmpy.inference import VariableElimination

In [ ]:
from discretization import discretize_data

columns_to_discretize = ['age', 'energy_level', 'oxygen_saturation']
train, test, _ = discretize_data(
    train_data,
    test_data,
    n_bins=5,
    strategy='uniform',
    columns_to_discretize=columns_to_discretize
)

In [ ]:
# 1️⃣ Structure learning (Tree Augmented Naive Bayes)
estimator = TreeSearch(train)
dag = estimator.estimate(estimator_type='tan', class_node='pulmonary_disease')

# 2️⃣ Wrap in DiscreteBayesianNetwork
model = DiscreteBayesianNetwork(dag.edges())

# 3️⃣ Parameter learning
model.fit(train, estimator=MaximumLikelihoodEstimator)

# 4️⃣ Inference
inference = VariableElimination(model)

In [ ]:
# 5️⃣ Prediction loop
from sklearn.metrics import accuracy_score, confusion_matrix
class_node = 'pulmonary_disease'
X_test = test.drop(columns=[class_node])
y_true = test[class_node]

y_pred = model.predict(X_test)
y_proba_df = model.predict_probability(X_test)
y_proba = y_proba_df['pulmonary_disease_1'].fillna(0.0).values

In [ ]:
custom_reports_from_proba(y_true, y_proba)

In [ ]:
# After structure learning
print("Number of edges:", len(dag.edges()))
print("Edges:", dag.edges())

# Check that it follows TAN structure
for node in dag.nodes():
    if node != 'pulmonary_disease':
        parents = list(dag.predecessors(node))
        print(f"{node} has {len(parents)} parents: {parents}")

In [ ]:
# 1️⃣ Aprendizaje de estructura
estimator = TreeSearch(train)
dag = estimator.estimate(
    estimator_type='tan',
    class_node='pulmonary_disease',
    edge_weights_fn='mutual_info'
)

model = DiscreteBayesianNetwork(dag.edges())

# 2️⃣ Aprendizaje de parámetros
model.fit(train, estimator=MaximumLikelihoodEstimator)